**Detecting the Card and Calculating its width**

In [13]:
import cv2
import numpy as np

def clahe(img, clip_limit=2.0, grid_size=(8,8)):
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)
    return clahe.apply(img)

src = cv2.imread("C://Users//tanus//OneDrive//Desktop//CREATIVITY//Cynaptics and ML//Jain_software//SLA task 1//In//img6.jpeg")

# HSV thresholding to get rid of as much background as possible
hsv = cv2.cvtColor(src.copy(), cv2.COLOR_BGR2HSV)
lower_blue = np.array([0, 0, 120])
upper_blue = np.array([180, 38, 255])
mask = cv2.inRange(hsv, lower_blue, upper_blue)
result = cv2.bitwise_and(src, src, mask=mask)
b, g, r = cv2.split(result)
g = clahe(g, 5, (3, 3))

# Adaptive Thresholding to isolate the card
img_blur = cv2.blur(g, (9, 9))
img_th = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 51, 2)

contours, hierarchy = cv2.findContours(img_th,
                                           cv2.RETR_CCOMP,
                                           cv2.CHAIN_APPROX_SIMPLE)

# Filter the rectangle by choosing only the big ones
# and choose the brightest rectangle as the card
max_brightness = 0
canvas = src.copy()
for cnt in contours:
    rect = cv2.boundingRect(cnt)
    x, y, w, h = rect
    if w*h > 40000 and w*h<200000:
        mask = np.zeros(src.shape, np.uint8)
        mask[y:y+h, x:x+w] = src[y:y+h, x:x+w]
        brightness = np.sum(mask)
        if brightness > max_brightness:
            brightest_rectangle = rect
            max_brightness = brightness
        # cv2.imshow("mask", mask)
        cv2.waitKey(0)

x, y, w, h = brightest_rectangle
print("The width of card is", w)
cv2.rectangle(canvas, (x, y), (x+w, y+h), (0, 255, 0), 5)
# cv2.imshow("canvas", canvas)
# cv2.imwrite("card_detected.jpg", canvas)
cv2.waitKey(0)

The width of card is 531


-1

The width of the card on forehead is 531 in terms of pixels.

**Detecting Pupils using dlib library**

In [2]:
!pip install dlib

In [3]:
!pip install imutils

In [4]:
import os
import numpy as np
import cv2
from skimage import io
import argparse
import dlib
import imutils

In [5]:
def pixelReader(img,startHorizontal,startVertical,height):
	'''
	Used to read specific pixel of given image.
	img = image 
	startHorizontal =  horizontal starting pixel value
	startVertical = vertical starting value
	height = maximum limit for pixel reading
	'''
	# list for satisfied pixels
	blackColour = []

	# for loops for traversing pixels
	for j in range(-int(height*1.5), 0):
		for i in range(startVertical- height,startVertical+int(height*1.5)):
			# setting lower bound for pixels, termination 
			blackLowerRange = [80, 50, 50]
			pixel = startHorizontal + j
			colorCI = img[int(pixel), i]
			if ((colorCI[0] <= blackLowerRange[0] and colorCI[1] <= blackLowerRange[1] and colorCI[2] <= blackLowerRange[2])):
				blackColour.append([int(pixel), i])
	return blackColour

In [6]:
def getFaceAttributeVector(image):
	'''
	Used to get the 68 facial attributes of face image.
	image: it is an image.
	'''
	# dlib shape predictor 
	predictorPath = "C://Users//tanus//OneDrive//Desktop//CREATIVITY//Cynaptics and ML//Jain_software//SLA task 1//shape_predictor_68_face_landmarks.dat"
	detector = dlib.get_frontal_face_detector()
	predictor = dlib.shape_predictor(predictorPath)
	dets = detector(image)

	# if dlib is able to detect the faces 
	image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
	for k, d in enumerate(dets):
		shape = predictor(image, d)

	# to get the 68 facial points detect from dlib into a list
	faceCoord = np.empty([68, 2], dtype = int)

	for b in range(68):
		faceCoord[b][0] = shape.part(b).x
		faceCoord[b][1] = shape.part(b).y

	return faceCoord

In [7]:
def getEyeCoordinates(image, faceCoord):
	'''
	Used to crop the eyes from image
	image: it is an image.
	faceCoord: array of facial landmarks coordinates
	'''
	# using facial points to crop the part of the eyes from image
	leftEye = image[int(faceCoord[19][1]):int(faceCoord[42][1]),int(faceCoord[36][0]):int(faceCoord[39][0])]
	rightEye = image[int(faceCoord[19][1]):int(faceCoord[42][1]),int(faceCoord[42][0]):int(faceCoord[45][0])]

	eyeLCoordinate = [int(faceCoord[37][0]+int((faceCoord[38][0]-faceCoord[37][0])/2)), int(faceCoord[38][1]+int((faceCoord[40][1]-faceCoord[38][1])/2))]
	eyeRCoordinate = [int(faceCoord[43][0]+int((faceCoord[44][0]-faceCoord[43][0])/2)), int(faceCoord[43][1]+int((faceCoord[47][1]-faceCoord[43][1])/2))]

	leftBlackPixel = pixelReader(image,eyeLCoordinate[1],eyeLCoordinate[0],int((faceCoord[38][0]-faceCoord[37][0])/2))
	rightBlackPixel = pixelReader(image, eyeRCoordinate[1], eyeRCoordinate[0], int((faceCoord[44][0]-faceCoord[43][0])/2))
	return leftEye, rightEye, leftBlackPixel, rightBlackPixel

In [8]:
def getPupilPoint(img, blackCoordinates, eyeTopPointX, eyeBottomPointY):
	'''
	Used to get the coordinates of the pupil 
	image: cropped eye image
	blackCoordinates: It is the array of the black color pixels inside the eyes
	eyeTopPointX: eye's starting coordinates horizontally
	eyeBottomPointY: eye's starting coordinates vertically
	'''
	# after getting the eyes pixels applying cv2 method to detect circle pixels which we can do using houghcircles 
	gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT, dp = 1,minDist = 5,
		param1=250,param2=10,minRadius=1,maxRadius=-1)
	
	# check if houghcircles has detect any circle or not
	if circles is not None:
		circles = np.uint16(np.around(circles))
		for i in circles[0,:1]:
			pupilPoint = [int(eyeTopPointX[0])+ i[0],int(eyeBottomPointY[1]) + i[1]]
	else:
		# if HoughCircles is unable to detect the circle than using eyes points to get pupil points 
		a = 0
		for j,k in blackCoordinates:
			if a == int(len(blackCoordinates)/2):
				pupilPoint = [k,j]
			a += 1
	return pupilPoint

**Detecting the pupil coordinates and saving the image to output folder while drawing a line between the pupils**

In [23]:
l=[]
r=[]
for imageName in os.listdir("C://Users//tanus//OneDrive//Desktop//CREATIVITY//Cynaptics and ML//Jain_software//SLA task 1//In//"):
	image = io.imread("C://Users//tanus//OneDrive//Desktop//CREATIVITY//Cynaptics and ML//Jain_software//SLA task 1//In//"+imageName)
	image = imutils.resize(image, width=1000)
	# getting the faceattribute vector from dlib
	faceVector = getFaceAttributeVector(image)

	# getting the eye points
	leftEye, rightEye, eyeLeftBlackPixels, eyeRightBlackPixels = getEyeCoordinates(image, faceVector)

	# getting pupilpoint for left eye
	# leftEye is the cropped part of face image
	leftEyeCoord, eyeBrowCoord = faceVector[36], faceVector[19]
	leftPupilPoint = getPupilPoint(leftEye, eyeLeftBlackPixels, leftEyeCoord, eyeBrowCoord); l=leftPupilPoint

	# getting pupilpoint for right eye
	# rightEye is the cropped part of face image
	rightEyeCoord = faceVector[42]
	rightPupilPoint = getPupilPoint(rightEye, eyeRightBlackPixels, rightEyeCoord, eyeBrowCoord); r=rightPupilPoint
	
	# drawing pupil points on image
	cv2.circle(image, tuple(leftPupilPoint), 5, (255, 0, 0), -1)
	cv2.circle(image, tuple(rightPupilPoint), 5, (255, 0, 0), -1)
	finalImage = cv2.cvtColor(image, cv2.COLOR_RGB2BGR); finalImage=cv2.line(finalImage, tuple(l), tuple(r), (0, 255, 0), thickness=5)
	cv2.imwrite("C://Users//tanus//OneDrive//Desktop//CREATIVITY//Cynaptics and ML//Jain_software//SLA task 1//Out//"+imageName,finalImage)
print(l, r)

[331, 546] [632, 548]


**Calculating the pupil distance using the pinhole camera assumption and known card dimension (8.56 cm width) and displaying it on line in centimeter**

In [19]:
PARALLEL_PLANE_SCALING_FACTOR=1.372

In [20]:
import math
def scaleit(l,r,w):
    pupil_dist_in_pixel=float(math.sqrt((l[0]-r[0])*(l[0]-r[0])+(l[1]-r[1])*(l[1]-r[1])))
    return float((pupil_dist_in_pixel*9)/w)*PARALLEL_PLANE_SCALING_FACTOR
print("The pupil distance in cm is", scaleit(l,r,w))

The pupil distance in cm is 6.999679935227999


**Writing the PD calculated on the line between pupils**

In [32]:
k=[]
k.append(int((l[0]+r[0])/2))
k.append(int((l[1]+r[1])/2))
img = cv2.imread("C://Users//tanus//OneDrive//Desktop//CREATIVITY//Cynaptics and ML//Jain_software//SLA task 1//Out//img6.jpeg")
img = cv2.putText(img, str(math.ceil(scaleit(l,r,w)))+"cm", tuple(k), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5, cv2.LINE_AA)
cv2.imwrite("C://Users//tanus//OneDrive//Desktop//CREATIVITY//Cynaptics and ML//Jain_software//SLA task 1//"+"result.jpg",img)

True